# Preparación del Dataset para el entrenamiento de Modelos ML

Tras la exploración del dataset, en este notebook se transforma el dataset de autenticación en bruto en un conjunto de datos estructurado y optimizado para el entrenamiento de modelos de aprendizaje automático orientados a la detección de accesos atípicos en entornos IAM.

Importación de las mismas librerías

In [5]:
import pandas as pd
import numpy as np
import gc

A continuación, se definen las columnas y los diferentes tipos de datos que hay en cada una de ellas, tal y como se han dividido durante el proceso de exploración.

In [6]:
columns = [
    "time",
    "src_user",
    "dst_user",
    "src_host",
    "dst_host",
    "auth_type",
    "logon_type",
    "auth_orientation",
    "auth_result"
]

dtypes = {
    "time": "int64",
    "src_user": "category",
    "dst_user": "category",
    "src_host": "category",
    "dst_host": "category",
    "auth_type": "category",
    "logon_type": "category",
    "auth_orientation": "category",
    "auth_result": "category"
}

Debido al elevado tamaño del dataset, el procesamiento se va a realizar de forma incremental mediante la lectura por bloques (chunks), para evitar la carga completa del conjunto de datos en memoria.

In [7]:
#cada chunk contendrá 500_000 registros
CHUNKSIZE = 500_000
TIME_WINDOW = "1h"

In [8]:
#variable para almacenar las futuras columnas o como vamos a querer almacenar los datos
db_rows = []
len(db_rows)

0

Fase de procesamiento por bloques y agregación

In [9]:
#Almacenamos en diferentes bloques del tamaño definido anteriormente el dataset
chunks = pd.read_csv(
    "auth.txt",
    sep=",",
    header=None,
    names=columns,
    dtype=dtypes, #Forzamos los tipos de datos para poder ahorrar memoria
    chunksize=CHUNKSIZE
)

#Lectura del dataset por bloques
for chunk in chunks:
    # Conversión temporal
    # Conversion de la columna time UNIX a un formato de fecha real
    chunk["dt"] = pd.to_datetime(chunk["time"], unit="s", errors="coerce")
    chunk = chunk.dropna(subset=["dt"])
    chunk = chunk.set_index("dt")
    
    # Creación de indicadores básicos

    #indicador de fallo de autenticación
    # Conversión del resultado de autenticación a un formato texto
    chunk["fail"] = (
        chunk["auth_result"]
        .astype(str)
        .str.lower()
        .eq("fail")
    )
    
    #Indicador de acceso not bussines hours (nbh)
    # Indicador de acceso fuera de horario (0-6)
    hours = chunk.index.hour
    chunk["nbhours"] = (hours >= 0) & (hours <= 6)

    
    #Creación de perfiles de comportamiento definidor por una ventana temporal y un usuario
    agg = chunk.groupby([
        pd.Grouper(freq=TIME_WINDOW),
        "src_user"
    ]).agg(
        total_events=("auth_result", "count"),
        failed_events=("fail", "sum"),
        dst_hosts=("dst_host", "nunique"),
        src_hosts=("src_host", "nunique"),
        nbhours_events=("nbhours", "sum")
    )
    
    #Se almacena el resultado
    db_rows.append(agg.reset_index())
    
    #Liberación de memoria
    del chunk, agg
    gc.collect()

In [10]:
#Construcción del dataset final
db_df = pd.concat(db_rows, ignore_index=True)
db_df.head()

,dt,src_user,total_events,failed_events,dst_hosts,src_hosts,nbhours_events
0,1970-01-01,ANONYMOUS LOGON@C1065,434,0,1,82,434
1,1970-01-01,ANONYMOUS LOGON@C1208,24,0,1,3,24
2,1970-01-01,ANONYMOUS LOGON@C1529,10,0,1,1,10
3,1970-01-01,ANONYMOUS LOGON@C1570,1,0,1,1,1
4,1970-01-01,ANONYMOUS LOGON@C1624,12,0,1,1,12


In [11]:
#Vamos a añadir un índice temporal numérico
#Un índice temporal numérico es una forma de representar el tiempo como un número continuo, en lugar de como una fecha/hora (datetime).
db_df = db_df.sort_values("dt")
db_df["t_index"] = (
    db_df["dt"] - db_df["dt"].min()
).dt.total_seconds() / 3600

db_df.head()

,dt,src_user,total_events,failed_events,dst_hosts,src_hosts,nbhours_events,t_index
0,1970-01-01,ANONYMOUS LOGON@C1065,434,0,1,82,434,0.0
4061,1970-01-01,U561@DOM1,17,0,3,4,17,0.0
4060,1970-01-01,U556@DOM1,20,0,5,4,20,0.0
4059,1970-01-01,U555@DOM1,49,0,9,10,49,0.0
4058,1970-01-01,U553@DOM1,30,0,6,7,30,0.0


In [12]:
db_df["dt"].head(5)

0      1970-01-01
4061   1970-01-01
4060   1970-01-01
4059   1970-01-01
4058   1970-01-01
Name: dt, dtype: datetime64[s]

In [13]:
db_df["dt"].dtype

dtype('<M8[s]')

In [14]:
db_df.loc[:5, "dt"].dt.hour

0       0
4061    0
4060    0
4059    0
4058    0
       ..
10      0
8       0
7       0
6       0
5       0
Name: dt, Length: 4056, dtype: int32

In [15]:
#Creación de métricas derivadas
#Se calcula la proporción de accesos fallidos
db_df["fail_ratio"] = (
    db_df["failed_events"] / db_df["total_events"]
)

#Se calcula la proporción de acceso durante las horas fuera de trabajo
db_df["nbhours_ratio"] = (
    db_df["nbhours_events"] / db_df["total_events"]
)

#Evitamos resultados NaN, que puedan molestar a los MLs
db_df = db_df.replace([np.inf, -np.inf], 0).fillna(0)


In [16]:
#Proceso de limpieza antes de terminar
db_df.describe()

#Eliminar usuarios con poca actividad o ventanas vacías
db_df = db_df[db_df["total_events"] > 1]

In [17]:
#Seleccionamos las columnas que tendra nuestor dataset
DB_COLS = [
    "dt",
    "src_user",
    "total_events",
    "failed_events",
    "fail_ratio",
    "dst_hosts",
    "src_hosts",
    "nbhours_events",
    "nbhours_ratio"
]

db_df[DB_COLS].to_csv(
    "lanl_db_feature.csv",
    index=False
)


El dataset preparado se almacena antes de aplicar técnicas de normalización, de modo que el escalado se realiza exclusivamente sobre los datos utilizados en cada uno de los experimentos. Este enfoque evita la introducción de fugas de información y preserva la interpretabilidad de las características.

In [18]:
#Normalización
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#db_scaled = scaler.fit_transform(db)

In [19]:
#db_scaled.shape

## Conclusión de la fase de preparación

En esta fase se ha transformado el conjunto de registros de autenticación en bruto en un dataset estructurado basado en perfiles de comportamiento de usuario agregados por ventanas temporales. La aplicación de técnicas de limpieza, agregación e ingeniería de características
ha permitido reducir significativamente el volumen de datos, manteniendo al mismo tiempo la información relevante para la detección de accesos atípicos.

El dataset resultante se encuentra normalizado y preparado para su utilización en modelos de aprendizaje automático no supervisado, que serán evaluados en la siguiente fase del trabajo.


En este trabajo se adopta un enfoque basado en perfiles de comportamiento, donde cada instancia del dataset representa la actividad de una identidad en una ventana temporal determinada. Esta estrategia permite capturar patrones de uso habituales y reducir el ruido inherente a los eventos individuales, facilitando la detección de desviaciones significativas asociadas a accesos atípicos. Asimismo, el uso de perfiles de comportamiento permite transformar registros de autenticación en bruto en un conjunto de características numéricas adecuadas para el entrenamiento de modelos de aprendizaje automático no supervisado.

En lugar de analizar eventos aislados, este trabajo modela el comportamiento de las identidades a lo largo del tiempo, permitiendo detectar accesos atípicos como desviaciones respecto a patrones normales de uso.